In [ ]:
from glob import glob
import pandas as pd
import string
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import openpyxl
from openpyxl.styles import Alignment, Font
from xlsxwriter import Workbook

In [ ]:
regions = ['conus', 'upper_midwest', 'northern_rockies_and_plains', 'west', 'southwest', 
           'south', 'southeast', 'northeast', 'northwest', 'ohio_valley']
regions = ['mro',
 'npcc',
 'northeast',
 'northern_rockies_and_plains',
 'northwest',
 'ohio_valley',
 'rf',
 'serc',
 'south',
 'southeast',
 'southern_company',
 'southwest',
 'tva',
 'texas_re',
 'upper_midwest',
 'wecc',
 'west',
 'conus',
 'atlantic', 'pacific', 'gulf']
regions = sorted(regions)

In [ ]:
metrics = ['ks_stat', 'bias_p95', 'bias_p50', 'bias_p5', 'percent_bias_p95', 'percent_bias_p50', 'percent_bias_p5']

In [ ]:
index = {
    'temperature_2m_ks_stat': 'T2M KS',
    'temperature_2m_bias_p50': 'T2M Bias P50 (°C)',
    'temperature_max_2m_ks_stat': 'T2M Max KS',
    'temperature_max_2m_bias_p95': 'T2M Max Bias P95 (°C)',
    'temperature_min_2m_ks_stat': 'T2M Min KS',
    'temperature_min_2m_bias_p5': 'T2M Min Bias P5 (°C)',
    
    'relativehumidity_2m_ks_stat': 'RH2M KS',
    'relativehumidity_2m_percent_bias_p50': 'RH2M Bias P50 (%)',
    'relativehumidity_max_2m_ks_stat': 'RH2M Max KS',
    'relativehumidity_max_2m_percent_bias_p95': 'RH2M Max Bias P95 (%)',
    'relativehumidity_min_2m_ks_stat': 'RH2M Min KS',
    'relativehumidity_min_2m_percent_bias_p5': 'RH2M Min Bias P5 (%)',
    
    'pr_ks_stat': 'PR KS',
    'pr_percent_bias_p50': 'PR Bias P50 (%)',
    'rsds_ks_stat': 'GHI KS',
    'rsds_percent_bias_p50': 'GHI Bias P50 (%)',
    'windspeed_100m_ks_stat': 'WS 100m KS',
    'windspeed_100m_percent_bias_p50': 'WS 100m Bias P50 (%)',
}


In [ ]:
weights = {
    'T2M KS': 1/6, 
    'T2M Bias P50 (°C)': 1/6, 
    'T2M Max KS': 1/6,
    'T2M Max Bias P95 (°C)': 1/6, 
    'T2M Min KS': 1/6, 
    'T2M Min Bias P5 (°C)': 1/6,
    'RH2M KS': 1/2, 
    'RH2M Bias P50 (%)': 1/2, 
    'RH2M Max KS': np.nan,
    'RH2M Max Bias P95 (%)': np.nan, 
    'RH2M Min KS': np.nan, 
    'RH2M Min Bias P5 (%)': np.nan,
    'PR KS': 1/2, 
    'PR Bias P50 (%)': 1/2, 
    'GHI KS': 1/2, 
    'GHI Bias P50 (%)': 1/2,
    'WS 100m KS': 1/2, 
    'WS 100m Bias P50 (%)': 1/2, 
    'Process Skill': 1, 
}

In [ ]:
# global and CONUS skills from https://doi.org/10.1029/2023JD039774
ncar_skills = pd.read_csv('./ncar/ncar_model_scores.csv', index_col=0)

In [ ]:
def get_region_df(region):
    df = None
    for metric in metrics:

        fp = f'./skill_summaries/skill_summary_{region}_{metric}.csv'
        idf = pd.read_csv(fp)
        idf['metric'] = metric
        idf['tag'] = idf['feature'] + '_' + idf['metric']
        idf = idf.drop(['feature', 'metric'], axis=1)
        idf['tag'] = idf['tag'].map(index)
        idf = idf.set_index('tag')

        if df is None:
            df = idf
        else:
            df = pd.concat((df, idf))

    df = df.loc[list(index.values())]
    df = df[sorted(df.columns)]
    return df

In [ ]:
coolwarm = plt.cm.get_cmap('coolwarm')

# Retrieve the red and blue colors from the 'coolwarm' colormap
red = coolwarm(1.0)  # Color at the beginning of the colormap
blue = coolwarm(0.0) # Color at the end of the colormap
white = coolwarm(0.5)

# Define the colors for the custom colormap using the retrieved colors
colors = [red, white, blue, white, red]  # R -> W -> B -> W -> R
positions = [0, 0.33, 0.5, 0.67, 1]  # Define the positions of the colors

# Create the custom colormap
coolwarm_sym = LinearSegmentedColormap.from_list("reflected_coolwarm", list(zip(positions, colors)))


In [ ]:
for region in regions:
# for region in ['conus']:
    df = get_region_df(region)
    df.index.name=None

    # add NCAR skills
    for model in df.columns:
        skill = ncar_skills.at[model, 'score']
        df.at['Process Skill', model] = skill
    
    all_scores = []
    for metric, row in df.iterrows():
        row = np.abs(row)
        score = (row - row.min()) / (row.max() - row.min())
        score *= weights[metric]
        all_scores.append(score)
    
    all_scores = pd.concat(all_scores, axis=1).T
    all_scores = all_scores.dropna()
    scores = all_scores.sum(0)
    
    ranks = scores.sort_values()
    df = df[ranks.index]
    
    style = [dict(selector="th.col_heading",
                 props=[("writing-mode", "vertical-rl"), 
                        ('transform', 'rotateZ(-180deg)'),
                        ('text-align', 'left'),
                       ]),
            dict(selector="th.row_heading",
                 props=[('text-align', 'right'),
                       ]),
            ]

    styled_df = df.style.set_table_styles(style, overwrite=False)

    styled_df = styled_df.background_gradient('viridis')
    for metric, row in df.iterrows():
        if 'KS' in metric or 'process' in metric.lower():
            styled_df = styled_df.background_gradient('coolwarm', subset=(metric,slice(None)), vmin=row.min(), vmax=row.max())
            styled_df.format('{:.3f}', subset=(metric, slice(None)))
        else: 
            vmax = np.maximum(np.abs(row.min()), np.abs(row.max()))
            styled_df = styled_df.background_gradient(coolwarm_sym, subset=(metric,slice(None)), vmin=-vmax, vmax=vmax)
            styled_df.format('{:.3f}', subset=(metric, slice(None)))

    styled_df.applymap(lambda x: 'font-size: 10px; width: 30px; height: 15px; text-align: center;')
    fpxl = f'skill_{region}.xlsx'
    fphtml = f'skill_{region}.html'
    styled_df.to_excel(fpxl, engine='xlsxwriter')
    
    wb = openpyxl.load_workbook(fpxl)
    ws = wb['Sheet1']

    ws.column_dimensions['a'].width = 20
    ws.row_dimensions[1].height = 80

    for row in range(2, 21):
        cell = ws[f'A{row}']
        cell.alignment = Alignment(horizontal='left', vertical='center')

    for alpha in list(string.ascii_lowercase)[1:]:
        cell = ws[f'{alpha}1']
        cell.alignment = Alignment(horizontal='center', vertical='bottom', textRotation=90)

    for alpha in list(string.ascii_lowercase)[1:]:
        ws.column_dimensions[alpha].width = 3
        for row in range(2, 21):
            max_in_row = np.max(np.abs([cell.value for cell in ws[row][1:] if cell.value is not None]))
            cell = ws[f'{alpha}{row}']
            if max_in_row > 15:
                cell.number_format = '0.0'
            else:
                cell.number_format = '0.00'

            cell.font = Font(size="6")
            cell.alignment = Alignment(horizontal='center', vertical='center')

    wb.save(fpxl)
    styled_df.to_html(fphtml)